In [17]:
import pandas as pd
#import matplotlib.pyplot as plt
#import descartes
import geopandas as gpd #cond install geopandas
#from shapely.geometry import Point, Polygon
import folium
import numpy as np
from folium.plugins import MarkerCluster


%matplotlib inline
#https://towardsdatascience.com/geopandas-101-plot-any-data-with-a-latitude-and-longitude-on-a-map-98e01944b972

In [3]:
df = pd.read_csv('Tripadvisordata_red.csv')
df[["Latitude","Longitude"]] = df['Location'].str.split(',',expand=True)
crs = {'init':'epsg:4326'}
df["Longitude"] = df["Longitude"].astype(float)
df["Latitude"] = df["Latitude"].astype(float)
df["Ranking on list"] = df["Ranking on list"].astype(int)
df["Ranking_color"] = ""
df["Dollar_sign_color"] = ""
df["Dollar_value_color"] = ""
df["Color_atmos"] = ""

In [4]:
df["Price Class Numeric"].describe()
df["Price Class Numeric"].nunique()
df["Price Class Numeric"].var()
df["Price Class Numeric"].mode()
df["Stemning"].unique()

array([nan, 45., 40., 50., 35., 30., 25., 20.])

<br><br>

In [5]:
df.head()

,Unnamed: 0,Name,Main rating,Ranking on list,Price range,Price class,Number of reviews,Location,Distance from Kgs. Nytorv,God pris,...,Service,Stemning,New Price class,Price Class Numeric,Latitude,Longitude,Ranking_color,Dollar_sign_color,Dollar_value_color,Color_atmos
0,0,Kokkeriet,4.5,13,------16770.-970.,--$$$$---,769 anmeldelser,"55.687626,12.585126",850.0,45.0,...,50.0,NaN,$$$$,4.0,55.687626,12.585126,,,,
1,1,The Olive Kitchen & Bar,5.0,3,------1674.-174.,--$$-$$$---------,2.138 anmeldelser,"55.68125,12.571284",820.0,45.0,...,50.0,NaN,$$-$$$,2.5,55.681250,12.571284,,,,
2,2,Pizzeria Mamemi & Wine Bar,5.0,7,"------16,,,",--$---------,927 anmeldelser,"55.667767,12.548116",2640.0,45.0,...,50.0,NaN,$,1.0,55.667767,12.548116,,,,
3,3,Zahida at Kellerdirk,5.0,5,------1647.-248.,--$$-$$$---------,878 anmeldelser,"55.67162,12.546196",2560.0,45.0,...,50.0,NaN,$$-$$$,2.5,55.671620,12.546196,,,,
4,4,Grams Lækkerier,5.0,1,------1633.-100.,--$---------,720 anmeldelser,"55.671036,12.562724",1670.0,45.0,...,50.0,45.0,$,1.0,55.671036,12.562724,,,,


<br><br>

In [25]:
#fuction that creates a color colum for dollar sign
color_list = ["#DE0000", "#EE7F00", "#FFFF00", "#B1D911", "#63B322"]

def dollar_color_class(x):
    
    if x == "$" or x == "$-$$":
        return color_list[4] #green
    elif x == "$$" or x == "$$-$$$":
        return color_list[3] #light green
    elif x == "$$$" or x == "$$$-$$$$":
        return color_list[2] #yellow
    elif x == "$$$$" or x == "$$$$-$$$$$":
        return color_list[1] #light orange 
    elif x == "$$$$$":
        return color_list[0] #red
    else:
        return "black" #'#00ff001C' #transparent

    
for x in df["New Price class"].index:
        df["Dollar_sign_color"][x] = dollar_color_class(df["Dollar_sign_color"][x])
        print(x)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

<br>

In [6]:
#fuction that creates a color colum for dollar value

def dollar_color_num(x):
    if x == 1:
        return '#00FF00' #green
    elif x == 1.5:
        return
    elif x == 2:
        return '#7CFF00' #light green
    elif x == 2.5:
        return
    elif x == 3:
        return '#F7FF00' #yellow
    elif x == 3.5:
        return
    elif x == 4:
        return '#FF6900' #light orange
    elif x == 4.5:
        return
    elif x == 5:
        return '#FF0000' #red
    elif x == 99:
        return 'black'
    else:
        return '#00ff001C' - transparent

    
for x in df["Price Class Numeric"].index:
        df["Dollar_value_color"][x] = dollar_color_num(df["Price Class Numeric"][x])
        #print(x)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
#fuction that creates a color colum for rating value

#https://www.strangeplanet.fr/work/gradient-generator/index.php
color_list = ["#DE0000", "#E43300", "#EB6600", "#F19900", "#F8CC00", "#FFFF00", "#DFEF06", "#C0E00D", "#A1D114", "#82C21B", "#63B322"]

def color_producer(x):
    
    if x < 200:
        return color_list[10] 
    elif x > 200 and x < 400:
        return color_list[9] 
    elif x > 400 and x < 600:
        return color_list[8] 
    elif x > 600 and x < 800:
        return color_list[7]  
    elif x > 800 and x < 1000:
        return color_list[6] 
    elif x > 1000 and x < 1200:
        return color_list[5] 
    elif x > 1200 and x < 1400:
        return color_list[4] 
    elif x > 1400 and x < 1600:
        return color_list[3] 
    elif x > 1600 and x < 1800:
        return color_list[2] 
    elif x > 1800 and x < 2000:
        return color_list[1] 
    elif x > 2000 and x < 2200:
        return color_list[0] 
    else:
        return 'black'

for x in df["Ranking on list"].index:
        print(x)
        df["Ranking_color"][x] = color_producer(df["Ranking on list"][x])

0
1


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [8]:
#def to atmosphere
color_list = ["#DE0000", "#E43300", "#EB6600", "#F19900", "#F8CC00", "#FFFF00", "#DFEF06", "#C0E00D", "#A1D114", "#82C21B", "#63B322"]

def atmos_producer(x):
    if x == 50:
        return color_list[8] 
    elif x == 45:
        return color_list[7] 
    elif x == 40:
        return color_list[6] 
    elif x == 35:
        return color_list[5]  
    elif x == 30:
        return color_list[4] 
    elif x == 25:
        return color_list[3] 
    elif x == 20:
        return color_list[2] 
    elif x == 15:
        return color_list[1] 
    elif x == 10:
        return color_list[0] 
    else:
        return 'black'

for x in df["Stemning"].index:
        print(x)
        df["Color_atmos"][x] = atmos_producer(df["Stemning"][x])

0
1
2

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278

<br><br><br><br>

# Color map of ranking

In [102]:
map_rest = folium.Map(location = [55.679977,12.5841893], zoom_start = 13)

In [103]:
mc = MarkerCluster()

for x,y,z,v,q in zip(df["Latitude"],df["Longitude"],df["Ranking_color"],df["Name"],df["Ranking on list"]):
    mc.add_child(folium.CircleMarker([x,y],radius=6, color = z, fill_color=z,fill=True,fill_opacity=1, tooltip = [v,q]))

In [104]:
map_rest.add_child(mc)
map_rest

<br><br><br><br>

# Color map of dollar sign 

In [27]:
map_dollar = folium.Map(location = [55.679977,12.5841893], zoom_start = 14)

In [28]:
for x,y,z,v in zip(df["Latitude"],df["Longitude"],df["Dollar_sign_color"],df["Name"]):
    folium.CircleMarker([x,y],radius=2, color = z, tooltip = v).add_to(map_dollar)

In [29]:
map_dollar

<br><br>
# Color map of dollar value 

In [16]:
map_dollar_value = folium.Map(location = [55.679977,12.5841893], zoom_start = 13)

In [17]:
for x,y,z,v in zip(df["Latitude"],df["Longitude"],df["Dollar_value_color"],df["Name"]):
    folium.CircleMarker([x,y],radius=2, color = z, tooltip = v).add_to(map_dollar_value)

In [18]:
map_dollar_value

<br> <br> <br>
# Color map of athmosphere


In [19]:
map_at = folium.Map(location = [55.679977,12.5841893], zoom_start = 13)

In [20]:
for x,y,z,v in zip(df["Latitude"],df["Longitude"],df["Color_atmos"],df["Name"]):
    folium.CircleMarker([x,y],radius=5, color = z, tooltip = v).add_to(map_at)

In [21]:
map_at